# Predicción del default usando SVC

Configuración de variables

In [ ]:
from sklearn.svm import SVC

test_path = '../files/input/test_data.csv.zip'
train_path = '../files/input/train_data.csv.zip'
categorical = ['SEX', 'EDUCATION', 'MARRIAGE']
estimator = SVC(random_state=11)
"""
param_grid = {
    'selector__k': [10, 15, 20, 'all'],
    'classifier__C': [0.1, 1, 5],
    'classifier__kernel': ['rbf'],
    'classifier__gamma': ['scale', 0.01, 0.1, 1, 'auto'],
}
"""
param_grid = {
    'selector__k': [15, 17, 20, 'all'],
    #'classifier__C': [10, 50],
    #'classifier__kernel': ['rbf', 'sigmoid'],
    'classifier__gamma': [0.01, 0.1, 1],
    #'classifier__degree': [1, 2, 3, 5, 10],
}

filepath_model = '../files/models'
name_model = 'model.pkl.gz'

## Paso 1.

Realice la limpieza de los datasets:

- Renombre la columna "default payment next month" a "default".
- Remueva la columna "ID".
- Elimine los registros con informacion no disponible.
- Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, agrupe estos valores en la categoría "others".
- Renombre la columna "default payment next month" a "default"
- remueva la columna "ID".

In [2]:
def clean_data(df):

    df = df.rename(columns = {'default payment next month': 'default'})     # Renombrar columna target
    df = df.drop(columns = ['ID'])                                          # Eliminar columna 'ID'
    df = df.dropna()                                                        # Eliminar filas con valores nulos
    df['EDUCATION'] = df['EDUCATION'].astype(int)                           # Convertir 'EDUCATION' a entero
    df.loc[df['EDUCATION'] > 4, 'EDUCATION'] = 4                            # Limitar 'EDUCATION' a 4
    df = df.query('MARRIAGE != 0 and EDUCATION != 0')                       # Filtrar filas

    return df

## Paso 2.

Divida los datasets en x_train, y_train, x_test, y_test.

In [3]:
def load_clean_data(test_path, train_path):
    import pandas as pd

    # Cargar datos
    train = pd.read_csv(train_path, index_col=False, compression='zip')
    test = pd.read_csv(test_path, index_col=False, compression='zip')

    # Limpiar datos
    train = clean_data(train)
    test = clean_data(test)

    # Separar características y target
    target = 'default'
    data_test = test.drop(columns=[target])
    data_train = train.drop(columns=[target])
    target_test = test[target].astype(int)
    target_train = train[target].astype(int)

    return data_train, target_train, data_test, target_test

## Paso 3.
Cree un pipeline para el modelo de clasificación. Este pipeline debe contener las siguientes capas:

- Transforma las variables categoricas usando el métodobone-hot-encoding.
- Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
- Estandariza la matriz de entrada.
- Selecciona las K columnas mas relevantes de la matrix de entrada.
- Ajusta una maquina de vectores de soporte (svm).

In [4]:
def make_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OneHotEncoder, StandardScaler
    from sklearn.decomposition import PCA
    from sklearn.compose import ColumnTransformer
    from sklearn.feature_selection import SelectKBest, f_classif
    from sklearn.svm import SVC

    X_train = load_clean_data(test_path, train_path)[0]
    numerical = [c for c in X_train.columns if c not in categorical]
    transformer = ColumnTransformer(transformers=[
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical),
        ('num', StandardScaler(), numerical)
    ], remainder='passthrough')

    pipeline = Pipeline(steps=[
        ('preprocessor', transformer),
        ('pca', PCA(n_components=None)),
        ('selector', SelectKBest(score_func=f_classif)),
        ('classifier', SVC(random_state=11))
        ], verbose=False
    )

    return pipeline

## Paso 4.
- Optimice los hiperparametros del pipeline usando validación cruzada.
- Use 10 splits para la validación cruzada. 
- Use la función de precision balanceada para medir la precisión del modelo.

In [5]:
def make_grid_search(pipeline, param_grid, n_folds = 10, n_jobs = -1):
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import make_scorer, balanced_accuracy_score, precision_score

    cv = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    scorers = {
        'accuracy': 'accuracy',
        'balanced_accuracy': make_scorer(balanced_accuracy_score),
        'precision': make_scorer(precision_score)
    }

    grid_search = GridSearchCV(estimator=pipeline,
                               param_grid=param_grid,
                               scoring='balanced_accuracy',
                               cv=cv,
                               n_jobs=n_jobs,
                               verbose=False)

    return grid_search

## Paso 5.

Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz". - Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.
#

In [6]:
def save_model(model, filepath = '../files/models'):

    import gzip
    import pickle
    import os

    os.makedirs(filepath, exist_ok=True)
    model_path = os.path.join(filepath, 'model.pkl.gz')

    with gzip.open(model_path, 'wb') as f:
        pickle.dump(model, f)

# ----------------------------------------------------------------------------

def load_model(filepath = '../files/models', name = 'model.pkl.gz'):
    import os
    import gzip
    import pickle

    model_path = os.path.join(filepath, name)

    if not os.path.exists(model_path):
        return None
    with gzip.open(model_path, 'rb') as f:
        model = pickle.load(f)

    return model

## Paso 6.

Calcule las metricas de precision, precision balanceada, recall, y f1-score para los conjuntos de entrenamiento y prueba.
Guardelas en el archivo files/output/metrics.json. Cada fila del archivo es un diccionario con las metricas de un modelo.
Este diccionario tiene un campo para indicar si es el conjunto de entrenamiento o prueba. Por ejemplo:

- {'type': 'metrics', 'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}

- {'type': 'metrics', 'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

Entrenamiento

In [7]:
def train_model(model):
    
    from sklearn.metrics import balanced_accuracy_score
    
    X_train, y_train, X_test, y_test = load_clean_data(test_path, train_path)
    
    model.fit(X_train, y_train)
    best_model = load_model()

    if best_model is not None:

        saved_bas = balanced_accuracy_score(
        y_true = y_test, y_pred = best_model.predict(X_test)
        )
        
        current_bas = balanced_accuracy_score(
        y_true = y_test, y_pred = model.predict(X_test)
        )

        if saved_bas >= current_bas:
            model = best_model
    
    save_model(model)

In [8]:
def train_svc_model(param_grid):

    pipeline = make_pipeline()
    
    model = make_grid_search(
        pipeline=pipeline, 
        param_grid=param_grid,
        )

    train_model(model)

Métricas

In [9]:
def eval_metrics(model, x_train, y_train, x_test, y_test):
        
    from sklearn.metrics import (
        precision_score,
        balanced_accuracy_score,
        recall_score,
        f1_score,
        confusion_matrix,
    )

    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    def metrics_dict(y_true, y_pred, dataset):
        return {
            "type": "metrics",
            "dataset": dataset,
            "precision": float(precision_score(y_true, y_pred, zero_division=0)),
            "balanced_accuracy": float(balanced_accuracy_score(y_true, y_pred)),
            "recall": float(recall_score(y_true, y_pred, zero_division=0)),
            "f1_score": float(f1_score(y_true, y_pred, zero_division=0)),
        }

    def cm_dict(y_true, y_pred, dataset):
        cm = confusion_matrix(y_true.astype(int), y_pred.astype(int), labels=[0, 1])
        tn, fp, fn, tp = int(cm[0, 0]), int(cm[0, 1]), int(cm[1, 0]), int(cm[1, 1])
        return {
            "type": "cm_matrix",
            "dataset": dataset,
            "true_0": {"predicted_0": tn, "predicted_1": fp},
            "true_1": {"predicted_0": fn, "predicted_1": tp},
        }

    # Construye los 4 renglones en el orden exacto que pide el test
    metrics_train = metrics_dict(y_train, y_pred_train, "train")
    metrics_test  = metrics_dict(y_test,  y_pred_test,  "test")
    cm_train      = cm_dict(y_train, y_pred_train, "train")
    cm_test       = cm_dict(y_test,  y_pred_test,  "test")

    return metrics_train, metrics_test, cm_train, cm_test


In [10]:
def save_report(
   metrics_train,
   metrics_test,
   cm_train,
   cm_test,
):
    
    import json
    import os
    
    path = '../files/output/metrics.json'

    if not os.path.exists(os.path.dirname(path)):
       os.makedirs(os.path.dirname(path))

    with open(path, 'w', encoding='utf-8', newline='\n') as f:
       f.write(json.dumps(metrics_train) + '\n')
       f.write(json.dumps(metrics_test) + '\n')
       f.write(json.dumps(cm_train) + '\n')
       f.write(json.dumps(cm_test) + '\n')

In [11]:
def print_report(metrics_train, metrics_test, cm_train=None, cm_test=None):
    """Imprime un resumen compacto de métricas.
    Muestra valores de test con el valor de train entre paréntesis.
    """
    def fmt(name, test_val, train_val):
        return f"{name:>20}: {test_val:.4f} ({train_val:.4f})"

    print("-" * 80)
    print("Metrics summary (test (train))")
    print("-" * 80)
    print(fmt("Balanced Accuracy", metrics_test["balanced_accuracy"], metrics_train["balanced_accuracy"]))
    print(fmt("Precision",         metrics_test["precision"],         metrics_train["precision"]))
    print(fmt("Recall",            metrics_test["recall"],            metrics_train["recall"]))
    print(fmt("F1-score",          metrics_test["f1_score"],          metrics_train["f1_score"]))
    if cm_test and cm_train:
        print("-" * 80)
        print("Confusion matrix (test):")
        print(f" true_0 -> predicted_0: {cm_test['true_0']['predicted_0']}, predicted_1: {cm_test['true_0']['predicted_1']}")
        print(f" true_1 -> predicted_0: {cm_test['true_1']['predicted_0']}, predicted_1: {cm_test['true_1']['predicted_1']}")
    print("-" * 80)

In [12]:
def check_estimator():

    x_train, y_train, x_test, y_test = load_clean_data(test_path, train_path)

    model = load_model()

    metrics_train, metrics_test, cm_train, cm_test = eval_metrics(
    model,
    x_train,
    y_train,
    x_test,
    y_test,
    )
    save_report(
        metrics_train,
        metrics_test,
        cm_train,
        cm_test,
    )
    print_report(             
        metrics_train,
        metrics_test,
        cm_train,
        cm_test,
    )


Mejores parámetros

In [13]:
def print_get_params():
    model = load_model()
    print("Get model parameters:")
    for param, value in model.get_params().items():
        print(f"  {param}: {value}")


def print_best_model_params():
    model = load_model()
    print("Best model parameters:")
    for param, value in model.best_params_.items():
        print(f"  {param}: {value}")

## EJECUCIÓN

In [14]:
train_svc_model(param_grid=param_grid)
check_estimator()

--------------------------------------------------------------------------------
Metrics summary (test (train))
--------------------------------------------------------------------------------
   Balanced Accuracy: 0.6680 (0.6650)
           Precision: 0.6752 (0.7030)
              Recall: 0.3861 (0.3763)
            F1-score: 0.4913 (0.4902)
--------------------------------------------------------------------------------
Confusion matrix (test):
 true_0 -> predicted_0: 6719, predicted_1: 354
 true_1 -> predicted_0: 1170, predicted_1: 736
--------------------------------------------------------------------------------


In [15]:
print_best_model_params()

Best model parameters:
  classifier__gamma: 0.1
  selector__k: 17


In [16]:
print_get_params()

Get model parameters:
  cv: StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
  error_score: nan
  estimator__memory: None
  estimator__steps: [('preprocessor', ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['SEX', 'EDUCATION', 'MARRIAGE']),
                                ('num', StandardScaler(),
                                 ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3',
                                  'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1',
                                  'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
                                  'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
                                  'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4',
                                  'PAY_AMT5', 'PAY_AMT6'])])), ('pca', PCA()), ('selector', SelectKBest()), (